<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>ChatGPT and Friends</h1>
<h1>GPT Models</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from datetime import datetime
import json

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

import openai
from openai import OpenAI

import termcolor
from termcolor import colored

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.12.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 21b9940cec1a0c4befc502f81feeba07c252d364

json      : 2.0.9
matplotlib: 3.8.0
pandas    : 2.1.4
tqdm      : 4.66.4
numpy     : 1.26.4
watermark : 2.4.3
openai    : 1.30.5
termcolor : 2.4.0



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Basic Usage

The first step is always to load up the API key from the local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key

In [4]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

We start by getting a list of supported models.

In [5]:
model_list = client.models.list().data

In total we have 33 models

In [6]:
len(model_list)

33

Along with some information about each model...

In [8]:
model_list[:3]

[Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'),
 Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'),
 Model(id='dall-e-2', created=1698798177, object='model', owned_by='system')]

But let's just get a list of model names

In [9]:
print("\n".join(sorted([model.id for model in model_list])))

babbage-002
dall-e-2
dall-e-3
davinci-002
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-0301
gpt-3.5-turbo-0613
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
gpt-4
gpt-4-0125-preview
gpt-4-0613
gpt-4-1106-preview
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4-turbo-preview
gpt-4o
gpt-4o-2024-05-13
gpt-4o-2024-08-06
gpt-4o-mini
gpt-4o-mini-2024-07-18
text-embedding-3-large
text-embedding-3-small
text-embedding-ada-002
tts-1
tts-1-1106
tts-1-hd
tts-1-hd-1106
whisper-1


## Basic Prompt

The recommended model for exploration is `gpt-4o`, so we'll stick with it for now. The basic setup is relatively straightforward:

In [10]:
%%time
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

CPU times: user 15.4 ms, sys: 3.51 ms, total: 18.9 ms
Wall time: 8.26 s


Which produces a response object

In [11]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

Which we can treat as a simple object

In [12]:
pprint(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's most well-known weakness is Kryptonite, a radioactive element from his home planet of Krypton. There are several different forms of Kryptonite, each having unique effects on Superman:\n\n- **Green Kryptonite**: This is the most commonly known form. Exposure to green Kryptonite weakens Superman, drains his powers, and can eventually kill him with prolonged exposure.\n- **Red Kryptonite**: Typically causes unpredictable physical changes or mental effects in Superman. Each exposure to red Kryptonite has a different effect and can last up to 48 hours.\n- **Gold Kryptonite**: Removes Superman's superpowers permanently upon contact.\n- **Blue Kryptonite**: Affects Bizarro, an imperfect clone of Superman, weakening him in the same way that green Kryptonite affects Superman.\n- **Black Kryptonite**: Can split a person into their good and evil halves.\n\nAside from Kryptonite, Superman also ha

The model answer can be found in the "message" named tuple inside the "choices" list

In [13]:
print(response.choices[0].message.content)

Superman's most well-known weakness is Kryptonite, a radioactive element from his home planet of Krypton. There are several different forms of Kryptonite, each having unique effects on Superman:

- **Green Kryptonite**: This is the most commonly known form. Exposure to green Kryptonite weakens Superman, drains his powers, and can eventually kill him with prolonged exposure.
- **Red Kryptonite**: Typically causes unpredictable physical changes or mental effects in Superman. Each exposure to red Kryptonite has a different effect and can last up to 48 hours.
- **Gold Kryptonite**: Removes Superman's superpowers permanently upon contact.
- **Blue Kryptonite**: Affects Bizarro, an imperfect clone of Superman, weakening him in the same way that green Kryptonite affects Superman.
- **Black Kryptonite**: Can split a person into their good and evil halves.

Aside from Kryptonite, Superman also has other vulnerabilities, albeit less pronounced:

- **Magic**: Superman's powers do not protect him 

To request multiple answers, we must include the `n` parameter with the number of answers we want

In [14]:
%%time
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "What are the different kinds of Kryptonite?"},
    ],
    n=3
)

CPU times: user 11.2 ms, sys: 2.11 ms, total: 13.3 ms
Wall time: 9.34 s


And we can access each of the answers individually int he choices list

In [15]:
for output in response.choices:
    print("==========")
    print(output.message.role.title()) 
    print("==========")
    print(output.message.content)
    print("==========\n")

Assistant
Kryptonite is most commonly known as the substance from Superman's home planet of Krypton that weakens him and other Kryptonians. While the green variety is the most well-known, there are actually several different kinds of Kryptonite in various forms of Superman media, each with unique properties. Here are some of the most notable types:

1. **Green Kryptonite**: The most iconic form, it weakens Superman and other Kryptonians, causing extreme pain and potentially death with prolonged exposure.

2. **Red Kryptonite**: Causes unpredictable and often bizarre changes in Kryptonians. Its effects vary widely, ranging from physical transformations to altered personalities.

3. **Blue Kryptonite**: Typically affects Bizarro, a flawed clone of Superman. In Bizarro's case, it weakens him just as green Kryptonite affects Superman.

4. **Gold Kryptonite**: Permanently removes a Kryptonian's superhuman abilities by destroying their ability to process yellow sunlight.

5. **White Kryptoni

In [16]:
response.usage

CompletionUsage(completion_tokens=1237, prompt_tokens=16, total_tokens=1253)

# Temperature

In [17]:
%%time
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=1.2
)

CPU times: user 11.9 ms, sys: 2.51 ms, total: 14.4 ms
Wall time: 11.7 s


In [18]:
print(response.choices[0].message.content)

Once upon a time, in a quaint village nestled between emerald hills, lived a curious girl named Elara. She had a heart full of wonder and a mind thirsty for adventure. Elara's favorite place was an ancient forest on the outskirts of the village that brimmed with history and whispered secrets through the rustling leaves.

One sunlit afternoon, as she wandered deeper into the forest than ever before, she stumbled upon a hidden hollow tree that emanated a soft, golden glow. Inside the hollow, she found an intricate, ornate key—bronze with delicate filigree and a tiny, ruby-red gemstone at its center. Intrigued, she decided to explore further and see if the mysterious key unlocked anything close by.

Following a narrow, winding path, Elara arrived at an old, moss-covered stone door. To her amazement, the key fit perfectly into the lock. With a deep breath, she turned it, and the heavy door creaked open to reveal a hidden grotto on the other side, filled with vibrant, luminescent flowers an

In [19]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=0
)

CPU times: user 9.92 ms, sys: 2.16 ms, total: 12.1 ms
Wall time: 4.01 s


In [22]:
print(response.choices[0].message.content)

Once upon a time, in a small village nestled in the mountains, there lived a young girl named Lily. Lily was known for her kindness and generosity, always willing to help those in need.

One day, a terrible storm hit the village, causing widespread damage and leaving many families homeless. Lily knew she had to do something to help. She gathered her friends and together they started a relief effort, collecting food, clothing, and supplies for those affected by the storm.

As word spread of Lily's efforts, more and more people in the village joined in, donating whatever they could to help their neighbors in need. Soon, the village was buzzing with activity as everyone worked together to rebuild and support those who had lost everything.

Thanks to Lily's leadership and the community's generosity, the village was able to recover from the storm stronger than ever before. And from that day on, Lily was known as a hero, not just in her village, but in the hearts of all who knew her.


## Few-shot prompting

We can also provide several examples of mappings between input and output.

In [23]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
        {"role": "assistant", "content": "Things working well together will increase revenue."},
        {"role": "user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)

This last-minute change means we can't do everything for the client's project.


# Formatted output

In [24]:
%%time
userInput = "blueberry pancakes"

prompt = """return a recipe for %s.
        Provide your response as a JSON object with the following schema:
        {"dish": "%s", "ingredients": ["", "", ...],
        "instructions": ["", "", ... ]}""" % (userInput, userInput)

response = client.chat.completions.create(
          model = "gpt-4o",
          messages = [
            { "role": "system", "content": "You are a helpful recipe assistant."},
            { "role": "user",   "content": prompt }
          ]
)

CPU times: user 10.4 ms, sys: 2.63 ms, total: 13 ms
Wall time: 5.36 s


In [26]:
print(prompt)

return a recipe for blueberry pancakes.
        Provide your response as a JSON object with the following schema:
        {"dish": "blueberry pancakes", "ingredients": ["", "", ...],
        "instructions": ["", "", ... ]}


In [27]:
json_output = response.choices[0].message.content

In [28]:
print(json_output)

{
  "dish": "blueberry pancakes",
  "ingredients": [
    "1 cup all-purpose flour",
    "1 tablespoon sugar",
    "1 teaspoon baking powder",
    "1/2 teaspoon baking soda",
    "1/4 teaspoon salt",
    "3/4 cup buttermilk",
    "1/4 cup milk",
    "1 large egg",
    "2 tablespoons unsalted butter, melted",
    "1 cup fresh or frozen blueberries",
    "Butter or oil for cooking"
  ],
  "instructions": [
    "In a large mixing bowl, whisk together the flour, sugar, baking powder, baking soda, and salt.",
    "In a separate bowl, combine the buttermilk, milk, egg, and melted butter. Whisk until well blended.",
    "Pour the wet ingredients into the dry ingredients and stir until just combined. Do not overmix; the batter should be slightly lumpy.",
    "Gently fold in the blueberries.",
    "Heat a griddle or large skillet over medium heat. Lightly grease with butter or oil.",
    "Pour 1/4 cup of batter onto the griddle for each pancake.",
    "Cook until bubbles form on the surface and 

In [29]:
output = json.loads(json_output)

In [30]:
output["ingredients"]

['1 cup all-purpose flour',
 '1 tablespoon sugar',
 '1 teaspoon baking powder',
 '1/2 teaspoon baking soda',
 '1/4 teaspoon salt',
 '3/4 cup buttermilk',
 '1/4 cup milk',
 '1 large egg',
 '2 tablespoons unsalted butter, melted',
 '1 cup fresh or frozen blueberries',
 'Butter or oil for cooking']

In [31]:
output["instructions"]

['In a large mixing bowl, whisk together the flour, sugar, baking powder, baking soda, and salt.',
 'In a separate bowl, combine the buttermilk, milk, egg, and melted butter. Whisk until well blended.',
 'Pour the wet ingredients into the dry ingredients and stir until just combined. Do not overmix; the batter should be slightly lumpy.',
 'Gently fold in the blueberries.',
 'Heat a griddle or large skillet over medium heat. Lightly grease with butter or oil.',
 'Pour 1/4 cup of batter onto the griddle for each pancake.',
 'Cook until bubbles form on the surface and the edges look set, about 2-3 minutes. Flip the pancake and cook until the other side is golden brown, another 2-3 minutes.',
 'Repeat with the remaining batter, greasing the griddle as needed.',
 'Serve warm with your favorite toppings such as maple syrup, butter, or additional blueberries.']

# Translation

In [32]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": "You're a professional English-Italian translator."}, 
              {"role": "user", "content": "Translate 'Be the change that you wish to see in the world.' into Italian"}],
    temperature=0,
)

In [33]:
response.choices[0].message.content

'"Sii il cambiamento che desideri vedere nel mondo."'

# Process unstructured information

Inspired by https://platform.openai.com/examples/default-parse-data

In [34]:
prompt = """There are many fruits that were found on the recently discovered planet Goocrux. 
There are neoskizzles that grow there, which are purple and taste like candy. There are also 
loheckles, which are a grayish blue fruit and are very tart, a little bit like a lemon. Pounits 
are a bright green color and are more savory than sweet. There are also plenty of loopnovas which 
are a neon pink flavor and taste like cotton candy. Finally, there are fruits called glowls, which 
have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them."""

In [35]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {"role": "system", 
         "content": "You will be provided with unstructured data, and your task is to parse it into CSV format."}, 
        {"role": "user", 
         "content": prompt}],
    temperature=0,
)

In [36]:
print(response.choices[0].message.content)

Sure, here is the data parsed into CSV format:

```csv
Fruit,Color,Taste
Neoskizzles,Purple,Candy
Loheckles,Grayish Blue,Tart
Pounits,Bright Green,Savory
Loopnovas,Neon Pink,Cotton Candy
Glowls,Pale Orange,Sour and Bitter
```


In [37]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "system", "content": """
            Read this paragraph 
            
            `%s` 
            
            and use it to answer some questions.""" % prompt}, 
              {"role": "user", "content": "What are pounits?"}],
    temperature=0,
)

In [38]:
print(response.choices[0].message.content)

Pounits are a bright green fruit found on the planet Goocrux. They are more savory than sweet.


<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>